In [ ]:
# jupyter-notebook 

import cv2
import numpy as np
from matplotlib import pyplot as plt
import itertools  
import time
import numba as nb
import copy
from components_container import ComponentsContainerWithHistory
import logging
logging.basicConfig(level=logging.INFO)
from skimage import io
%matplotlib inline
from skimage.morphology import binary_dilation, reconstruction
from skimage.measure import label
from scipy import ndimage

In [ ]:
#!pip install opencv-python numba


In [ ]:
displayed_sq = 1500
width_margin = 0
height_margin = 0
def show_img(img):
    fig = plt.figure(figsize=(13, 13), dpi= 80, facecolor='w', edgecolor='k')
    plt.imshow(img[height_margin:height_margin+displayed_sq,width_margin:width_margin+displayed_sq], cmap="ocean")

In [ ]:
#INimg = io.imread('../../notebook/3155_Glia_6M.tif')
#INimg = io.imread('../../notebook/3026_Glia_6M.tif')
INimg = io.imread('../../notebook/2997_Glia_6M.tif')
#INimg = io.imread('../../notebook/3280_Glia_6M.tif')
#INimg = io.imread('../../notebook/1873_Glia_6M.tif')
#INimg = io.imread('../../notebook/1796_Glia_6M.tif')

INimg = np.moveaxis(INimg,0,1)

#img = img[:2500,:2500]

In [ ]:
INimg.shape

In [ ]:
def draw_grayscale(container):
    labs = sorted(container.nucleus_labs, key = lambda x: -container.components_index.get(x).size)
    print(container.markers.shape)
    pic = np.zeros(container.markers.shape)
    nucl_color = 1
    axon_color = 1
    for nucl_lab in labs:
        pic[container.markers == nucl_lab] = nucl_color#np.sqrt(container.components_index.get(nucl_lab).size)
        for ax in container.components_index.get(nucl_lab).axons:
            pic[container.markers == ax.label] = nucl_color#np.sqrt(container.components_index.get(nucl_lab).size)

        nucl_color += 1

    
        
    #fig = plt.figure(figsize=(11, 11), dpi=80, facecolor='w', edgecolor='k')
    
    #plt.imshow(pic[height_margin:height_margin + displayed_sq, width_margin:width_margin + displayed_sq],
    #               cmap="Set1")
    print(nucl_color)
    return pic, nucl_color

def computeMask(img, BACKGROUND_INTENSITY_THRESHOLD=30, MIN_AXON_SIZE=150,show=False):
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = img.copy()
    invert = cv2.bitwise_not(gray)
    if show:
        show_img(invert)
    blur = cv2.GaussianBlur(img,(5,5),0)
    ret, binary = cv2.threshold(blur, BACKGROUND_INTENSITY_THRESHOLD, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernel = np.ones((1, 1), 'uint8')

    binary = cv2.dilate(binary, kernel, iterations=2)
    if show:
        show_img(binary)
    # filter all noise using minimum size threshold
    container = ComponentsContainerWithHistory(binary, MIN_AXON_SIZE, logging.INFO)
    if show:
        show_img(container.markers)
    # merging close components based on centroids and contour distance

    # for correct work please recompile from here if any of following parameters changed

    PREMERGE_CENTROIDS_DISTANCE_T  = 30
    PREMERGE_CONTOUR_DISTANCE_T = 50

    container.merge_components_closer_than(PREMERGE_CENTROIDS_DISTANCE_T, PREMERGE_CONTOUR_DISTANCE_T)
    if show:
        print("Showing merged components")
        show_img(container.markers)
        container.show_components()
    #splitting nucleus and axons based on size

    MIN_NUCLEUS_SIZE = 150

    container.split_nucl_axon(MIN_NUCLEUS_SIZE)
    container.add_hull_info_to_nucleus()
    if show:
        container.show_nucl_and_axons()
    #grouping possible axons to nucleus

    CENTROIDS_DISTANCE_T = 30
    CONTOUR_DISTANCE_T = 50

    container.group_axons_to_nucleus(CENTROIDS_DISTANCE_T, CONTOUR_DISTANCE_T)
    #container.show_nucl_and_axons()
    #invert_3 = np.stack((invert,invert,invert), axis=-1)
    #pic = enhance_img_with_container(invert_3, container)
    #cv2.imwrite('../test.png',pic)
    return draw_grayscale(container)


def enhance_img_with_container(img, container):
    edge_color = [80,0,80]
    hull_color = [0,0,100]
    for nucl_lab in container.nucleus_labs:           
        hull = np.array(container.components_index.get(nucl_lab).hull, np.int32)
        hull.reshape((-1,1,2))
        hull = hull[:,::-1]
        cv2.polylines(img, [hull],True, hull_color)

        for ed in container.components_index.get(nucl_lab).edge:
            img[ed[0], ed[1]] = edge_color
    return img



def saveMask(outFile,oMask):
    #io.imsave(, outMask.astype('uint32'), plugin='tifffile')
    #convert to binary
    glNew = oMask>0
    #fill holes
    #dilate it 'dil' times
    for i in range(1):
        glNew = binary_dilation(glNew)
    glNew = ndimage.binary_fill_holes(glNew)

    # label image regions
    glNew = label(glNew)
    #print(glNew.shape)
    io.imsave(outFile,  np.moveaxis(glNew,0,1).astype('uint32'), plugin='tifffile')

In [ ]:
outMask  =   np.zeros((INimg.shape[0],INimg.shape[1]))
numCells = 0
outFile = '2997_Glia6M'
#outFile = '3280_Glia_6M'

w = INimg.shape[0]
h = INimg.shape[1]
TILE_SIZE = 3000
xNum = INimg.shape[0]//TILE_SIZE 
yNum = INimg.shape[1]//TILE_SIZE
print(w,h,TILE_SIZE, xNum, yNum)
for ix in range(xNum+1):
    for iy in range(yNum+1):
        xLim = (ix*TILE_SIZE,min(w,(ix+1)*TILE_SIZE))
        yLim = (iy*TILE_SIZE,min(h,(iy+1)*TILE_SIZE))
        print("DIMS....",ix,iy,xLim,yLim)
        #INimg[xLim[0]:xLim[1],yLim[0]:yLim[1]]
        #maskImg, Ncomp = computeMask(INimg[xLim[0]:xLim[1],yLim[0]:yLim[1],:], show=True)
        maskImg, Ncomp = computeMask(INimg[xLim[0]:xLim[1],yLim[0]:yLim[1]], show=False)
        print(Ncomp)
        outMask[xLim[0]:xLim[1],yLim[0]:yLim[1]] = maskImg
        numCells += Ncomp
        saveMask('../3155/'+str(ix)+'_'+str(iy)+outFile+'.tif', outMask[xLim[0]:xLim[1],yLim[0]:yLim[1]])
        #break
    #break

#saveMask('../'+outFile+'.tif', outMask)


In [ ]:
saveMask('../'+outFile+'.tif', outMask)
